# Purpose
This notebook will use the BLSTM model defined in the other notebook, with less epochs.

It will go through all the features, remove them one by one, and then train in the resulting model with n-1 features.

It will compare the resulting model, and store the corresponding mean and variance.

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, Input, TimeDistributed

from model.scoring_metrics import get_windiff, get_pk, get_k_kappa

from model_trainer_and_tester import read_in_dataset_lstm, test_set_evaluate_multiple_lstm
from tensorflow import keras

In [5]:
# First, finding the listing of what all the features are
temp_df = pd.read_csv('../results_merged_f0_stds_fixed/Bed002.csv', sep=';')

temp_df.columns

Index(['segID', 'StartTimeA', 'EndTimeA', 'StartTimeB', 'EndTimeB', 'pause',
       'speakerChange', 'similarity', 'boundary', 'f0_diff',
       'f0_baseline_diff', 'f0_stds_means'],
      dtype='object')

In [6]:
# Some of the features aren't really relevant, so I'll just go ahead and ignore them
# It's mainly the segID and the start and end times
all_features = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff', 'f0_stds_means']

results_dict = {}

# These are parameters that are constant regardless of the model
batch_size = 64
shifts = [-2, -1, 0, 1, 2]
hidden_units = 200

for elem in all_features:
    temp_features = all_features.copy()
    temp_features.remove(elem)
    n_timesteps = len(shifts)
    feature_count = len(temp_features)

    X_train, Y_train = read_in_dataset_lstm(temp_features, shifts, to_read='train')

    sample_weight = np.ones(shape=(len(Y_train),))
    # I'm gonna increase the weight by the inverse of the proportion of weird examples that there are
    # How I define if there is a weird sample is by summing along the 2D squares to find where there's a 1, and then does a sum of times there's a 1
    # I'm going to do n_timesteps times the inverse count frequency, because in the final version we only predict with the center value. So to correct for this I add this increase
    new_weight = n_timesteps*len(Y_train)/np.sum(Y_train, axis=1).sum()

    # Have to do a flatten() inside because of weird numpy stuff with a length 1 dimension
    sample_weight[(np.sum(Y_train, axis=1) >= 1).flatten()] = new_weight

    model = Sequential()
    # For the input number of units, I'll assume that number of timesteps * features is a good enough value
    model.add(Bidirectional(LSTM(hidden_units, activation='tanh', return_sequences=True, dropout=0.3), input_shape=(n_timesteps, feature_count)))
    model.add(Bidirectional(LSTM(hidden_units, activation='tanh', return_sequences=True, dropout=0.3), input_shape=(n_timesteps, feature_count)))

    model.add(Bidirectional(LSTM(hidden_units, activation='sigmoid', return_sequences=True, dropout=0.3)))
    # This last time distributed is super important, it follows the output structure of the paper I've been following closely
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))

    import tensorflow_addons as tfa
    model.compile(loss='binary_crossentropy', optimizer='RMSprop',
                  metrics=[keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')],
                  weighted_metrics=[]
                  )

    # train the model
    print('Fitting model')
    history = model.fit(X_train, Y_train,
                        batch_size=batch_size,
                        epochs=10,
                        #class_weight= {0:1, 1:10},
                        # sample_weight_mode='temporal',
                        sample_weight=sample_weight,
                        validation_split=0.1,
                        verbose=0
                        )

    temp_results = test_set_evaluate_multiple_lstm(model, temp_features, shifts)

    results_dict['No ' + elem] = pd.concat([temp_results.mean().add_suffix('_mean'), temp_results.std().add_suffix('_std')])
    print("Finished no " + elem)


Fitting model
86/86 [==============================] - 1s 10ms/step
Finished no pause
Fitting model
86/86 [==============================] - 1s 8ms/step
Finished no speakerChange
Fitting model
86/86 [==============================] - 1s 12ms/step
Finished no similarity
Fitting model
86/86 [==============================] - 1s 16ms/step
Finished no f0_diff
Fitting model
86/86 [==============================] - 1s 16ms/step
Finished no f0_baseline_diff
Fitting model
86/86 [==============================] - 1s 14ms/step
Finished no f0_stds_means


In [7]:
results_df = pd.DataFrame(results_dict)
results_df

,No pause,No speakerChange,No similarity,No f0_diff,No f0_baseline_diff,No f0_stds_means
Pk_mean,0.404572,0.417271,0.420344,0.423923,0.430957,0.455519
K-k_mean,0.000000,0.022989,0.037718,0.038008,0.015353,-0.030214
Windiff_mean,0.404572,0.431815,0.455550,0.460035,0.463814,0.474070
Pk_std,0.080020,0.107657,0.101914,0.114404,0.106259,0.131174
K-k_std,0.000000,0.170217,0.172378,0.158995,0.163749,0.204513
Windiff_std,0.080020,0.116347,0.104720,0.121257,0.111288,0.133676


In [8]:
results_df.to_csv('BLSTM_feature_results.csv')

In [9]:
all_features = ['pause', 'speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff', 'f0_stds_means']

results_dict_one_feature = {}

# These are parameters that are constant regardless of the model
batch_size = 64
shifts = [-2, -1, 0, 1, 2]
hidden_units = 200

for elem in all_features:
    temp_features = [elem]
    n_timesteps = len(shifts)
    feature_count = len(temp_features)

    X_train, Y_train = read_in_dataset_lstm(temp_features, shifts, to_read='train')

    sample_weight = np.ones(shape=(len(Y_train),))
    # I'm gonna increase the weight by the inverse of the proportion of weird examples that there are
    # How I define if there is a weird sample is by summing along the 2D squares to find where there's a 1, and then does a sum of times there's a 1
    # I'm going to do n_timesteps times the inverse count frequency, because in the final version we only predict with the center value. So to correct for this I add this increase
    new_weight = n_timesteps*len(Y_train)/np.sum(Y_train, axis=1).sum()

    # Have to do a flatten() inside because of weird numpy stuff with a length 1 dimension
    sample_weight[(np.sum(Y_train, axis=1) >= 1).flatten()] = new_weight

    model = Sequential()
    # For the input number of units, I'll assume that number of timesteps * features is a good enough value
    model.add(Bidirectional(LSTM(hidden_units, activation='tanh', return_sequences=True, dropout=0.3), input_shape=(n_timesteps, feature_count)))
    model.add(Bidirectional(LSTM(hidden_units, activation='tanh', return_sequences=True, dropout=0.3), input_shape=(n_timesteps, feature_count)))

    model.add(Bidirectional(LSTM(hidden_units, activation='sigmoid', return_sequences=True, dropout=0.3)))
    # This last time distributed is super important, it follows the output structure of the paper I've been following closely
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))

    import tensorflow_addons as tfa
    model.compile(loss='binary_crossentropy', optimizer='RMSprop',
                  metrics=[keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')],
                  weighted_metrics=[]
                  )

    # train the model
    print('Fitting model')
    history = model.fit(X_train, Y_train,
                        batch_size=batch_size,
                        epochs=10,
                        #class_weight= {0:1, 1:10},
                        # sample_weight_mode='temporal',
                        sample_weight=sample_weight,
                        validation_split=0.1,
                        verbose=0
                        )

    temp_results = test_set_evaluate_multiple_lstm(model, temp_features, shifts)

    results_dict_one_feature['Only ' + elem] = pd.concat([temp_results.mean().add_suffix('_mean'), temp_results.std().add_suffix('_std')])
    print("Finished no " + elem)

Fitting model
86/86 [==============================] - 1s 13ms/step
Finished no pause
Fitting model
86/86 [==============================] - 1s 14ms/step
Finished no speakerChange
Fitting model
86/86 [==============================] - 1s 12ms/step
Finished no similarity
Fitting model
86/86 [==============================] - 1s 12ms/step
Finished no f0_diff
Fitting model
86/86 [==============================] - 2s 18ms/step
Finished no f0_baseline_diff
Fitting model
86/86 [==============================] - 1s 10ms/step
Finished no f0_stds_means


In [10]:
results_df = pd.DataFrame(results_dict_one_feature)
results_df

,Only pause,Only speakerChange,Only similarity,Only f0_diff,Only f0_baseline_diff,Only f0_stds_means
Pk_mean,0.426238,0.404572,0.404572,0.404572,0.404572,0.404572
K-k_mean,0.010894,0.000000,0.000000,0.000000,0.000000,0.000000
Windiff_mean,0.439552,0.404572,0.404572,0.404572,0.404572,0.404572
Pk_std,0.116182,0.080020,0.080020,0.080020,0.080020,0.080020
K-k_std,0.163755,0.000000,0.000000,0.000000,0.000000,0.000000
Windiff_std,0.114746,0.080020,0.080020,0.080020,0.080020,0.080020


In [11]:
results_df.to_csv('BLSTM_only_one_feature_results.csv')